In [ ]:
import pandas as pd

FILES = {
    "EDILE": "Prezzario_EDILE_Veneto_2025_rev1.xlsx",
    "ELETTRICO": "Prezzario_ELETTRICO_Veneto_2025.xlsx",
    "TERMO": "Prezzario_TERMOIDRAULICA_Veneto_2025.xlsx",
    "STAT": "all_4923_1_Dati_STAT_02-POE-Materiali_edile1_OTT20.xlsx"
}

CANONICAL_CATEGORIES = {
    # Finiture
    "PAVIMENTO": ["gres", "piastrel", "porcellanato"],
    "RIVESTIMENTO": ["rivestimento", "maiolica"],
    "PARQUET": ["parquet"],
    "LAMINATO": ["laminato"],
    "BATTISCOPA": ["battiscopa"],

    # Finiture pareti
    "PITTURA": ["pittura", "idropittura", "smalto"],
    "INTONACO": ["intonaco"],
    "CARTONGESSO": ["cartongesso"],

    # Posa e preparazione
    "COLLANTE": ["collante"],
    "STUCCO_FUGHE": ["stucco", "fug"],
    "IMPERMEABILIZZAZIONE": ["imperme", "guaina"],
    "MASSETTO": ["massetto"],
    "CEMENTO": ["cemento"],
    "MALTA": ["malta"],

    # Impianto idraulico
    "TUBO_PEX": ["pex", "multistrato"],
    "TUBO_PVC": ["pvc"],

    # Sanitari
    "WC": ["wc", "vaso"],
    "LAVABO": ["lavabo"],
    "DOCCIA": ["doccia"],

    # Impianto elettrico
    "CAVO_ELETTRICO": ["cavo"],
    "PRESA": ["presa"],
    "INTERRUTTORE": ["interruttore"],
    "MAGNETOTERMICO": ["magnetotermico", "differenziale"],
    "CORPO_ILLUMINANTE": ["plafoniera", "lampada"],

    # Riscaldamento
    "RADIATORE": ["radiatore"],
    "CALDAIA": ["caldaia"],
    "VENTILAZIONE": ["ventilatore", "aspiratore"]
}



def assign_category(description: str):
    description = description.lower()
    for category, keywords in CANONICAL_CATEGORIES.items():
        for kw in keywords:
            if kw in description:
                return category
    return None


def extract_materials_from_file(path, source_name):
    xls = pd.ExcelFile(path)

    # Try to auto-detect correct sheet (choose the largest sheet)
    sheet_sizes = {}
    for sheet in xls.sheet_names:
        df_temp = pd.read_excel(path, sheet_name=sheet)
        sheet_sizes[sheet] = len(df_temp)

    main_sheet = max(sheet_sizes, key=sheet_sizes.get)

    df = pd.read_excel(path, sheet_name=main_sheet)
    df.columns = df.columns.astype(str).str.strip()

    # Detect columns dynamically
    code_col = next((c for c in df.columns if "codice" in c.lower()), None)
    desc_col = next((c for c in df.columns if "descr" in c.lower()), None)
    um_col = next((c for c in df.columns if "u.m" in c.lower() or c.lower() in {"um"}), None)
    price_col = next((c for c in df.columns if "prezzo" in c.lower()), None)

    if not all([code_col, desc_col, um_col, price_col]):
        print(f"Skipping {source_name} - missing required columns")
        return pd.DataFrame()

    df = df.rename(columns={
        code_col: "Codice",
        desc_col: "Descrizione",
        um_col: "U.M.",
        price_col: "PREZZO"
    })[["Codice", "Descrizione", "U.M.", "PREZZO"]]

    # Fix merged hierarchy
    df["Codice"] = df["Codice"].ffill()
    df["Descrizione"] = df["Descrizione"].ffill()

    # Clean price formatting
    s = df["PREZZO"].astype(str).str.strip()

    both = s.str.contains(",", na=False) & s.str.contains(r"\.", na=False)
    s.loc[both] = s.loc[both].str.replace(",", "", regex=False)

    only_comma = s.str.contains(",", na=False) & ~s.str.contains(r"\.", na=False)
    s.loc[only_comma] = s.loc[only_comma].str.replace(",", ".", regex=False)

    df["PREZZO"] = pd.to_numeric(s, errors="coerce")

    # Remove invalid rows
    df = df.dropna(subset=["PREZZO", "U.M."])

    df["Descrizione"] = df["Descrizione"].astype(str).str.lower().str.strip()
    df["U.M."] = df["U.M."].astype(str).str.lower().str.strip()

    # Remove labor
    df = df[~df["U.M."].str.contains("ora", na=False)]

    # Assign category
    df["CATEGORY"] = df["Descrizione"].apply(assign_category)

    # Keep only categorized interior materials
    df = df[df["CATEGORY"].notna()]

    df["SOURCE"] = source_name

    return df.reset_index(drop=True)


all_materials = []

for name, path in FILES.items():
    df_clean = extract_materials_from_file(path, name)
    print(f"{name} categorized rows:", len(df_clean))
    all_materials.append(df_clean)

materials_db = pd.concat(all_materials, ignore_index=True)

print("\nTotal unified categorized materials:", len(materials_db))
materials_db.head(5)

EDILE categorized rows: 444
ELETTRICO categorized rows: 1217
TERMO categorized rows: 132
STAT categorized rows: 143

Total unified categorized materials: 1936


,Codice,Descrizione,U.M.,PREZZO,CATEGORY,SOURCE
0,2. 05. 50. 40. 10,essenza di pino 20 - 85 per rivestimento paret...,m2,33.137038,RIVESTIMENTO,EDILE
1,2. 05. 50. 40. 20,essenza di larice 20 - 85 per rivestimento par...,m2,44.182718,RIVESTIMENTO,EDILE
2,2. 05. 50. 40. 30,essenza di pino 21 - 75 per rivestimento paret...,m2,38.574911,RIVESTIMENTO,EDILE
3,2. 05. 50. 40. 40,essenza di larice 21 - 75 per rivestimento par...,m2,52.371949,RIVESTIMENTO,EDILE
4,2. 30. 30. 10. 30,mattoni trafilati lisci o sabbiati per rivesti...,cad,0.791520,RIVESTIMENTO,EDILE


In [ ]:
import numpy as np

def build_quality_tiers(df):
    tier_rows = []

    for category, group in df.groupby("CATEGORY"):
        group = group.sort_values("PREZZO").reset_index(drop=True)

        if len(group) < 3:
            continue  # require enough items

        n = len(group)

        std_index = int(n * 0.3)
        prem_index = int(n * 0.7)

        std_row = group.iloc[[std_index]].copy()
        prem_row = group.iloc[[prem_index]].copy()

        std_row["QUALITA"] = "BASE"
        prem_row["QUALITA"] = "SUPERIORE"

        tier_rows.append(std_row)
        tier_rows.append(prem_row)

    return pd.concat(tier_rows, ignore_index=True)

In [ ]:
tiered_materials = build_quality_tiers(materials_db)

print("Tiered materials count:", len(tiered_materials))
tiered_materials.head(5)

Tiered materials count: 46


,Codice,Descrizione,U.M.,PREZZO,CATEGORY,SOURCE,QUALITA
0,6. 8. 20.143. 20,battiscopa in multistrato impiallicciato anieg...,m,5.547876,BATTISCOPA,EDILE,BASE
1,5. 14. 20. 10.100,"posa a colla (compresa) e fuga, di battiscopa",m,8.541347,BATTISCOPA,EDILE,SUPERIORE
2,3.610. 09,strumentazione per temperatura: - termostato p...,cad,101.684815,CALDAIA,TERMO,BASE
3,3.720.140. 50,e) caldaia a basamento tipo a condensazione fu...,cad,224.531934,CALDAIA,TERMO,SUPERIORE
4,3. 70. 00. 10. 40,rifiuti di cantiere come cartongesso materiali...,kg,0.250000,CARTONGESSO,EDILE,BASE


In [ ]:
tiered_materials['Descrizione'].head(1).values[0]

"battiscopa in multistrato impiallicciato aniegre' verniciato  coprente a base acqua ral 9010 sezione dim mm 70 x 10. posa in opera esclusa ("

In [ ]:
tiered_materials.to_csv("tiered_materials.csv", index=False)

In [18]:
import pandas as pd
df = pd.read_csv("tecnomat_catalog_onepage_per_leaf.csv")
df = df.drop(columns=['leaf_url','product_url','price_net','price_gross','image_url','currency','sku'])
df.head(1)

,level1,level2,level3,product_name,price_numeric,unit,availability
0,"Porte, finestre, infissi e falegnameria",Porte interne,Porte battenti,PORTA DA INTERNO BATTENTE MOON NEW ZANINI 80X2...,119.0,NaN,Disponibile


In [32]:
import re
import pandas as pd

INDOOR_L1_ALLOW = {
    "Pavimenti e rivestimenti",
    "Sanitari e bagno",
    "Idraulica",
    "Elettricità e fotovoltaico",
    "Illuminazione",
    "Porte, finestre, infissi e falegnameria",
    "Edilizia",
    "Ferramenta",
    "Riscaldamento e climatizzazione",
}

# words that usually mean "not interior renovation materials"
OUTDOOR_OR_NOISE_PAT = re.compile(
    r"(estern|giardin|piscin|terraz|balcon|garage|cancell|serrand|fotovoltaic|antenne|satellit|parabol|"
    r"cantiere|pontegg|trabattell|betonier|carriol|rampe|scale da lavoro|"
    r"manutenzione e pulizia piscina|lampade solari)",
    re.IGNORECASE
)

def filter_scraped_indoor(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()

    # Basic: keep only allowed level1
    d = d[d["level1"].isin(INDOOR_L1_ALLOW)]

    # Build a combined text for filtering
    text = (
        d["level1"].fillna("").astype(str) + " " +
        d["level2"].fillna("").astype(str) + " " +
        d["level3"].fillna("").astype(str) + " " +
        d.get("title", pd.Series([""]*len(d))).fillna("").astype(str)  # if you have title/name
    )

    # Remove outdoor/noise
    d = d[~text.str.contains(OUTDOOR_OR_NOISE_PAT, na=False)]

    # Optional: keep only available-ish products
    if "availability" in d.columns:
        d = d[d["availability"].isin(["Disponibile", "Disponibile a breve", "Disponibile su ordinazione"])]

    return d.reset_index(drop=True)

In [46]:
CANON_MAP_BY_TAXONOMY = [
    # Pavimenti & rivestimenti
    ("PAVIMENTO",        lambda r: "Pavimenti in" in str(r["level2"]) or "Pavimenti" in str(r["level2"])),
    ("RIVESTIMENTO",     lambda r: "Rivestimenti" in str(r["level2"]) or "Rivestimenti" in str(r["level3"])),
    ("BATTISCOPA",       lambda r: "Battiscopa" in str(r["level2"]) or "Battiscopa" in str(r["level3"])),
    ("COLLANTE",         lambda r: "Colle per pavimenti" in str(r["level3"]) or "Adesivi" in str(r["level3"])),
    ("STUCCO_FUGHE",     lambda r: "Stucchi" in str(r["level3"])),
    ("MASSETTO",         lambda r: "Massetti" in str(r["level3"])),
    ("IMPERMEABILIZZAZIONE", lambda r: "Impermeabilizzazione" in str(r["level2"])),

    # Bagno
    ("WC",               lambda r: "Sanitari" in str(r["level2"]) and ("wc" in str(r["level3"]).lower() or "copriwater" in str(r["level3"]).lower())),
    ("LAVABO",            lambda r: "Lavabi" in str(r["level2"]) or "Lavabi" in str(r["level3"])),
    ("DOCCIA",            lambda r: "Docce" in str(r["level2"]) or "Box doccia" in str(r["level3"]) or "Piatti doccia" in str(r["level3"])),

    # Idraulica / riscaldamento
    ("TUBO_PEX",          lambda r: "multistrato" in str(r.get("title","")).lower() or "pex" in str(r.get("title","")).lower()),
    ("TUBO_PVC",          lambda r: "pvc" in str(r.get("title","")).lower()),
    ("RADIATORE",         lambda r: "Termosifoni" in str(r["level2"]) or "termoarredi" in str(r["level2"]).lower()),
    ("CALDAIA",           lambda r: "Caldaie" in str(r["level2"])),

    # Elettrico
    ("PRESA",             lambda r: "prese" in str(r["level2"]).lower()),
    ("INTERRUTTORE",      lambda r: "interruttori" in str(r["level2"]).lower()),
    ("CAVO_ELETTRICO",    lambda r: "cavi" in str(r["level2"]).lower()),
    ("CORPO_ILLUMINANTE", lambda r: "Illuminazione" in str(r["level1"])),
]

def assign_category_scraped(row):
    text = " ".join([
        str(row.get("level1", "")),
        str(row.get("level2", "")),
        str(row.get("level3", "")),
        str(row.get("product_name", ""))
    ]).lower()

    # PAVIMENTI
    if "paviment" in text or "gres" in text or "parquet" in text or "laminat" in text:
        return "PAVIMENTO"

    # RIVESTIMENTI
    if "rivestiment" in text or "mosaico" in text:
        return "RIVESTIMENTO"

    # BATTISCOPA
    if "battiscopa" in text:
        return "BATTISCOPA"

    # COLLANTE
    if "colle" in text or "adesiv" in text:
        return "COLLANTE"

    # STUCCO
    if "stucc" in text or "fug" in text:
        return "STUCCO_FUGHE"

    # IMPERMEABILIZZAZIONE
    if "impermeabil" in text or "guaina" in text:
        return "IMPERMEABILIZZAZIONE"

    # WC
    if "wc" in text or "sanitari a terra" in text or "sanitari sospesi" in text:
        return "WC"

    # LAVABO
    if "lavab" in text:
        return "LAVABO"

    # DOCCIA
    if "doccia" in text or "box doccia" in text:
        return "DOCCIA"

    # TUBI
    if "pex" in text or "multistrato" in text:
        return "TUBO_PEX"

    if "pvc" in text:
        return "TUBO_PVC"

    # RADIATORE
    if "termosifon" in text or "termoarred" in text:
        return "RADIATORE"

    # CALDAIA
    if "caldaia" in text:
        return "CALDAIA"

    # PRESA
    if "presa" in text:
        return "PRESA"

    # INTERRUTTORE
    if "interruttor" in text:
        return "INTERRUTTORE"

    # CAVO
    if "cavo" in text:
        return "CAVO_ELETTRICO"

    # ILLUMINAZIONE
    if "plafonier" in text or "lampad" in text or "farett" in text:
        return "CORPO_ILLUMINANTE"

    return None

In [47]:
def prepare_scraped_for_engine(scraped_df):

    d = scraped_df.copy()

    # 1. Indoor filter
    d = filter_scraped_indoor(d)

    # 2. Assign category
    d["CATEGORY"] = d.apply(assign_category_scraped, axis=1)
    d = d[d["CATEGORY"].notna()]

    # 3. Keep only safe categories
    SAFE_CATEGORIES = {
        "PAVIMENTO": "m2",
        "RIVESTIMENTO": "m2",
        "BATTISCOPA": "m",
        "TUBO_PEX": "m",
        "TUBO_PVC": "m",
        "CAVO_ELETTRICO": "m",
        "WC": "cad",
        "LAVABO": "cad",
        "DOCCIA": "cad",
        "PRESA": "cad",
        "INTERRUTTORE": "cad",
        "RADIATORE": "cad",
        "CALDAIA": "cad",
        "CORPO_ILLUMINANTE": "cad"
    }

    d = d[d["CATEGORY"].isin(SAFE_CATEGORIES.keys())]

    # 4. Assign canonical unit
    d["U.M."] = d["CATEGORY"].map(SAFE_CATEGORIES)

    # 5. Normalize price
    d["PREZZO"] = pd.to_numeric(d["price_numeric"], errors="coerce")
    d = d.dropna(subset=["PREZZO"])

    d["SOURCE"] = "SCRAP_RETAIL"

    return d[["CATEGORY", "PREZZO", "U.M.", "SOURCE"]].reset_index(drop=True)

In [48]:
filtered_df = filter_scraped_indoor(df)
scraped_engine_df = prepare_scraped_for_engine(filtered_df)
print(len(scraped_engine_df))
print(scraped_engine_df.head())

/tmp/ipython-input-401/3852414632.py:39: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  d = d[~text.str.contains(OUTDOOR_OR_NOISE_PAT, na=False)]
/tmp/ipython-input-401/3852414632.py:39: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  d = d[~text.str.contains(OUTDOOR_OR_NOISE_PAT, na=False)]


2617
   CATEGORY  PREZZO U.M.        SOURCE
0  TUBO_PVC   16.95    m  SCRAP_RETAIL
1  TUBO_PVC   53.90    m  SCRAP_RETAIL
2        WC   12.50  cad  SCRAP_RETAIL
3        WC   12.50  cad  SCRAP_RETAIL
4        WC   11.90  cad  SCRAP_RETAIL
